# (WIP) PyTorch Cheatsheet

Some of the most commonly used commands/setups in PyTorch.

* TK - extra resources
* TK - website with sources
* TK - PyTorch best practices
* TK - how to get help, one of the best ways to search "pytorch how to make a convolutional neural network" or "pytorch transformer layers" or "pytorch loss functions"


# Topics to cover

* Imports
    * Domain libraries
    * Vision
    * Text
    * Audio 
* Device-agnostic code (CPU, GPU, MPS)
* Basics
    * Seeds
* Neural networks
    * Activation functions 
    * Transformers
    * CNN's
    * RNN's
* Training loop
* Testing loop
* Optimizers
* Loss functions
* Evaluation

* Extras


# Imports

In [1]:
import torch

# Check the version
print(f"PyTorch version: {torch.__version__}")

PyTorch version: 1.12.1+cu113


In [2]:
# Can also import the common abbreviation "nn" for "Neural Networks"
from torch import nn

# Almost everything in PyTorch is called a "Module" (you build neural networks by stacking together Modules)
this_is_a_module = nn.Linear(in_features=1,
                             out_features=1)
print(type(this_is_a_module))

<class 'torch.nn.modules.linear.Linear'>


### Data imports

Since most of machine learning is finding patterns in data, it's good to know how to work with datasets in PyTorch.

In [3]:
# Import PyTorch Dataset (you can store your data here) and DataLoader (you can load your data here)
from torch.utils.data import Dataset, DataLoader

## Domain libraries

Depending on the specific problem you're working on, PyTorch has several domain libraries.

- **[TorchVision](https://pytorch.org/vision/stable/index.html)** — PyTorch’s resident computer vision library. 
- **[TorchText](https://pytorch.org/text/stable/index.html)** — PyTorch’s in-built domain library for text.
- [**TorchAudio**](https://pytorch.org/audio/stable/index.html) — PyTorch’s domain library for everything audio. 
- **[TorchRec](https://pytorch.org/torchrec/)** — PyTorch’s newest in-built domain library for powering recommendation engines with deep learning.

### Computer Vision 

In [4]:
# Base computer vision library
import torchvision

# Other components of TorchVision (premade datasets, pretrained models and image transforms)
from torchvision import datasets, models, transforms 

### Text and Natural Language Processing 

In [5]:
# Base text and natural language processing library
import torchtext

# Other components of TorchText (premade datasets, pretrained models and text transforms)
from torchtext import datasets, models, transforms

### Audio and Speech

In [6]:
# Base audio and speech processing library
import torchaudio

# Other components of TorchAudio (premade datasets, pretrained models and text transforms)
from torchaudio import datasets, models, transforms

### Recommendation systems

**Note:** This library is currently in beta release, see the [GitHub page for installation](https://github.com/pytorch/torchrec#installation). 

In [7]:
# # Base recommendation system library 
# import torchrec

# # Other components of TorchRec
# from torchrec import datasets, models

## Device-agnostic code (using PyTorch on CPU, GPU or MPS)

Much of deep learning involves computing on tensors. 

Computing on tensors generally happens much faster on GPUs (graphics processing units, typically from NVIDIA) than CPUs (computer processing units).

MPS stands for "Metal Performance Shader" which is Apple's GPU (M1, M1 Pro, M2 etc).

It is advised to perform training on the fastest piece of hardware you have available, which will generally be: NVIDIA GPU (`"cuda"`) > MPS device (`"mps"`) > CPU (`"cpu"`).

* TK - See here for more: https://www.learnpytorch.io/00_pytorch_fundamentals/#2-getting-pytorch-to-run-on-the-gpu 
* MPS backend - https://pytorch.org/docs/stable/notes/mps.html 
* CUDA - https://pytorch.org/docs/stable/cuda.html 

**Note:** It is advised to setup device-agnostic code at the start of your workflow.

In [9]:
# Setup device-agnostic code 
if torch.cuda.is_available():
    device = "cuda" # NVIDIA GPU
elif torch.backends.mps_is_available():
    device = "mps" # Apple GPU
else:
    device = "cpu"

print(f"Using device: {device}")

Using device: cuda


### Sending a tensor to target device

In [12]:
# Create a tensor 
x = torch.tensor([1, 2, 3]) 
print(x.device) # defaults to CPU 

# Send tensor to target device
x = x.to(device)
print(x.device) 

cpu
cuda:0


## Setting random seeds

A lot of machine learning and deep learning involves taking random numbers in tensors and then shaping those random numbers to find/represent patterns in real data. 

However, sometimes you'll want "reproducible" randomness.

To do so, you can set the random seeds, see [Reproducibility (trying to take the random out of random)](https://www.learnpytorch.io/00_pytorch_fundamentals/#reproducibility-trying-to-take-the-random-out-of-random) for more.

In [16]:
import torch

# Set the random seed (you can set this to any number you like, it will "flavour"
# the randomness with that number.
torch.manual_seed(42)

# Create two random tensors
random_tensor_A = torch.rand(3, 4)

torch.manual_seed(42) # set the seed again (try commenting this out and see what happens)
random_tensor_B = torch.rand(3, 4)

print(f"Tensor A:\n{random_tensor_A}\n")
print(f"Tensor B:\n{random_tensor_B}\n")
print(f"Does Tensor A equal Tensor B? (anywhere)")
random_tensor_A == random_tensor_B

Tensor A:
tensor([[0.8823, 0.9150, 0.3829, 0.9593],
        [0.3904, 0.6009, 0.2566, 0.7936],
        [0.9408, 0.1332, 0.9346, 0.5936]])

Tensor B:
tensor([[0.8823, 0.9150, 0.3829, 0.9593],
        [0.3904, 0.6009, 0.2566, 0.7936],
        [0.9408, 0.1332, 0.9346, 0.5936]])

Does Tensor A equal Tensor B? (anywhere)


tensor([[True, True, True, True],
        [True, True, True, True],
        [True, True, True, True]])

You can also set the random seed on the GPU (CUDA devices).

In [18]:
# Set random seed on GPU
torch.cuda.manual_seed(42)

## Neural Networks

PyTorch has a very comprehensive library of pre-built neural network components (many of these are referred to as "modules" in the PyTorch ecosystem).

At a fundamental level neural networks are stacks of *layers*. Each of these layers performs some kind of operation on an input and produces an output.

How these layers stack together will dependent on the problem you're working on.

It's one of the most active areas of research in machine learning: how to stack neural network layers together.

The vast majority of neural network components in PyTorch are contained within the [`torch.nn` package](https://pytorch.org/docs/stable/nn.html) (`nn` is short for neural networks).

In [21]:
from torch import nn

### Linear layers

TK - See more: https://pytorch.org/docs/stable/nn.html#linear-layers 

In [19]:
# Create a linear layer with 10 in features and out features
linear_layer = nn.Linear(in_features=10,
                         out_features=10)
linear_layer

Linear(in_features=10, out_features=10, bias=True)

In [20]:
identity_layer = nn.Identity()
identity_layer

Identity()

### Convolutional layers (for making Convolutional Neural Networks)

Naming of convolutional layers usually follows `torch.nn.ConvXd` where `X` can be a value of `1`, `2` or `3`.

The `X` value represents the number of dimensions the convolution will operate over, for example, `1` for singular dimension text, `2` for two dimension images (height x width) and `3` for 3D objects such as video (video is considered a series of images with a time dimension, height x width x time).

* TK - See more: https://pytorch.org/docs/stable/nn.html#convolution-layers
* TK - See more: https://www.learnpytorch.io/03_pytorch_computer_vision/#7-model-2-building-a-convolutional-neural-network-cnn 

In [23]:
# Create a Conv1d layer (often used for text with a singular dimension)
conv1d = nn.Conv1d(in_channels=1,
                   out_channels=10,
                   kernel_size=3)
conv1d

Conv1d(1, 10, kernel_size=(3,), stride=(1,))

In [26]:
# Create a Conv2d layer (often used for images with Height x Width dimensions)
conv2d = nn.Conv2d(in_channels=3, # 3 channels for color images (red, green, blue)
                   out_channels=10,
                   kernel_size=3)
conv2d                   

Conv2d(3, 10, kernel_size=(3, 3), stride=(1, 1))

In [25]:
# Create a Conv3d layer (often used for video with Height x Width x Time dimensions)
conv3d = nn.Conv3d(in_channels=3,
                   out_channels=10,
                   kernel_size=3)
conv3d

Conv3d(3, 10, kernel_size=(3, 3, 3), stride=(1, 1, 1))

### TK - Transformer Layers

See more: https://pytorch.org/docs/stable/nn.html#transformer-layers

### TK - RNN's

See more: https://pytorch.org/docs/stable/nn.html#recurrent-layers

### TK - Activation Functions 

See more: (non-linear) - https://pytorch.org/docs/stable/nn.html#non-linear-activations-weighted-sum-nonlinearity 

### TK - Loss Functions

* TK - Also called cost function, critertion

See more: https://pytorch.org/docs/stable/nn.html#loss-functions 

### TK - Optimizers 

* TK - contained within the `torch.optim` package: https://pytorch.org/docs/stable/optim.html 

See more: 

## TK - End-to-end example workflow

TK - see workflow notebook

* create data
* create neural network
* create loss function/optimizer
* create training loop
* create testing loop
* evaluate/plot